In [26]:
import tensorflow as tf
import numpy as np

In [40]:
tf.reset_default_graph()

In [41]:
# For convenience
def seqtostr(input): 
    return ''.join(ix_to_char[ch] for ch in input[:])

In [42]:
# Kapathy's test function
def sample(x, n, cell, Why, by):
    W = cell[0]
    Wxh = W[:vocab_size]
    Whh = W[vocab_size:]
    bh = cell[1]
    h = np.zeros((1, hidden_size))
    ixes = []

    for step in range(n):
        h = np.tanh(np.dot(x, Wxh) + np.dot(h, Whh) + bh)
        y = np.dot(h, Why) + by
        # p = np.exp(y) / np.sum(np.exp(y))
        # ix = np.random.choice(list(range(vocab_size)), p=p.ravel())
        ix = np.argmax(y)
        x = np.zeros((1, vocab_size))
        x[0][ix] = 1
        ixes.append(ix)
    return ixes


In [43]:
# Karpathy's preprocessing
data = "hihello"

chars= list(set(data))
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }



In [44]:
# Preparing training
inputs = [[char_to_ix[ch] for ch in data[:-1]]]
targets = [[char_to_ix[ch] for ch in data[1:]]]
X_train = tf.one_hot(inputs, vocab_size)
Y_train = tf.one_hot(targets, vocab_size)

In [45]:
# Hyperparameters
hidden_size = 10
cost_val = 100 # initial cost
learning_rate = 1e-3 #0.001

In [46]:
# etc.
batch_size = 1
seq_length = len(data)-1


In [47]:

# Placeholder
X = tf.placeholder(tf.float32, [None, seq_length, vocab_size])
Y = tf.placeholder(tf.int32, [None, seq_length, vocab_size])

In [48]:
# Output layer of char-rnn
Why = tf.Variable(tf.random_normal([batch_size, hidden_size, vocab_size], name='weight_hy'))
by = tf.Variable(tf.random_normal([batch_size, 1, vocab_size]), name='bias_y')

In [49]:
# Setting cell, loss function, optimizer
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=hidden_size)
state = cell.zero_state(batch_size, tf.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=X,
                                   initial_state=state, dtype=tf.float32)
logit = tf.matmul(outputs, Why) + by
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=Y))
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [37]:
# hihello 를 list로 바꾸면 중복제거되서 shape가 5
print(X_train,Y_train)

Tensor("one_hot:0", shape=(1, 240, 28), dtype=float32) Tensor("one_hot_1:0", shape=(1, 240, 28), dtype=float32)


In [52]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())    
    for step in range(1001):
        #Test
        if step % 50 == 0:
            X_test = sess.run(X_train[0][0]).reshape(1, vocab_size) # Start from the beginning
            predtxt = sample(X_test, seq_length, sess.run(cell.variables), sess.run(Why), sess.run(by))
            print('step :', step, 'cost :', cost_val,
                  '\npred :', data[0] + seqtostr(predtxt), '\n','-'*50)
            
        # Train
        cost_val, _ = sess.run([cost, train], feed_dict={X:sess.run(X_train), Y:sess.run(Y_train)})

step : 0 cost : 0.19925487 
pred : hiieeii 
 --------------------------------------------------
step : 50 cost : 1.0066944 
pred : hieelii 
 --------------------------------------------------
step : 100 cost : 0.5478621 
pred : hihello 
 --------------------------------------------------
step : 150 cost : 0.31220034 
pred : hihello 
 --------------------------------------------------
step : 200 cost : 0.1961207 
pred : hihello 
 --------------------------------------------------
step : 250 cost : 0.13218275 
pred : hihello 
 --------------------------------------------------
step : 300 cost : 0.09447693 
pred : hihello 
 --------------------------------------------------
step : 350 cost : 0.0708718 
pred : hihello 
 --------------------------------------------------
step : 400 cost : 0.0552492 
pred : hihello 
 --------------------------------------------------
step : 450 cost : 0.04439844 
pred : hihello 
 --------------------------------------------------
step : 500 cost : 0.03655117